In [ ]:
!pip install transformers sentencepiece tensorflow datasets rouge_score


     |████████████████████████████████| 3.3 MB 4.0 MB/s 
     |████████████████████████████████| 1.2 MB 63.1 MB/s 
     |████████████████████████████████| 298 kB 65.0 MB/s 
     |████████████████████████████████| 596 kB 63.7 MB/s 
     |████████████████████████████████| 895 kB 80.1 MB/s 
     |████████████████████████████████| 3.3 MB 76.0 MB/s 
     |████████████████████████████████| 61 kB 606 kB/s 
     |████████████████████████████████| 1.1 MB 71.8 MB/s 
     |████████████████████████████████| 243 kB 95.7 MB/s 
     |████████████████████████████████| 132 kB 91.0 MB/s 
     |████████████████████████████████| 271 kB 95.0 MB/s 
     |████████████████████████████████| 160 kB 87.9 MB/s 
     |████████████████████████████████| 192 kB 72.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip show transformers

Name: transformers
Version: 4.12.5
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: /usr/local/lib/python3.7/dist-packages
Requires: numpy, filelock, tqdm, importlib-metadata, sacremoses, requests, packaging, regex, tokenizers, huggingface-hub, pyyaml
Required-by: 


In [ ]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [ ]:
# %env WANDB_PROJECT=nlp_final
%load_ext autoreload
%autoreload 2

In [ ]:
# Imports
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torchtext.data.metrics import bleu_score
import pandas as pd
import time
import numpy as np
import math
from tqdm import tqdm
tqdm.pandas()

# try %matplotlib notebook (if following doesn't work)
%matplotlib inline
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip3 install pickle5
import pickle5 as pickle

     |████████████████████████████████| 256 kB 4.1 MB/s 


# Loading Dataset

In [ ]:
with open('SVAMP.pkl', "rb") as fh:
  data_df = pickle.load(fh)

In [ ]:
print(data_df.shape)
data_df.sample(5)

(1000, 5)


,Body,Question,Equation,Answer,Type
543,A book has 3 chapters. The first chapter is 91...,How many more pages does the first chapter hav...,( 91.0 - 23.0 ),68.0,Subtraction
790,Zachary did 46 push-ups and 58 crunches in gym...,How many more crunches than push-ups did Zacha...,( 58.0 - 46.0 ),12.0,Subtraction
196,Jake has 8 fewer peaches and 10 more apples th...,How many more peaches than apples does Steven ...,( 18.0 - 11.0 ),7.0,Subtraction
514,Tiffany was collecting cans for recycling. On ...,How many bags did she find after monday?,( 10.0 + 4.0 ),14.0,Addition
215,Mary is baking a cake. The recipe calls for 5 ...,How many more cups of sugar does she need to add?,( 14.0 - 2.0 ),12.0,Subtraction


In [ ]:
from dataloader import *

In [ ]:
train_df, val_df = split_train_val(data_df, props=[0.85, 0.15], shuffle=True)
# view train val size
print(train_df.shape, val_df.shape)

(850, 5) (150, 5)


In [ ]:
# dialogue length distribution
token_lens = data_df.Question.progress_apply(lambda x: len(x)) + data_df.Body.progress_apply(lambda x: len(x))
token_lens.describe(percentiles=[0.5, 0.75, 0.90])

100%|██████████| 1000/1000 [00:00<00:00, 458494.10it/s]


count    1000.000000
mean      161.493000
std        44.822148
min        67.000000
50%       153.000000
75%       189.000000
90%       220.000000
max       330.000000
dtype: float64

In [ ]:
MAX_LEN = 60

lang = initLang(train_df[['Question', 'Equation', 'Body']])

initializing: Question | total_vocab_size: 63
initializing: Equation | total_vocab_size: 69
initializing: Body | total_vocab_size: 74


# Training Models

In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModelForSeq2SeqLM
  
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

In [ ]:
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
pegasus_tokenizer = PegasusTokenizer.from_pretrained(model_name)
pegasus_model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
t5_tokenizer = AutoTokenizer.from_pretrained("valhalla/t5-small-qg-hl")

t5_model = AutoModelForSeq2SeqLM.from_pretrained("valhalla/t5-small-qg-hl")

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/627 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [ ]:
t5_pegasus_small_tokenizer = AutoTokenizer.from_pretrained('imxly/t5-pegasus-small')

t5_pegasus_small_model = AutoModelForSeq2SeqLM.from_pretrained('imxly/t5-pegasus-small')

Downloading:   0%|          | 0.00/551 [00:00<?, ?B/s]

OSError: ignored

In [ ]:
mt5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

mt5_model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

In [ ]:
mbart_tokenizer = AutoTokenizer.from_pretrained("sshleifer/tiny-mbart")

mbart_model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/tiny-mbart")

In [ ]:
bart_tokenizer = AutoTokenizer.from_pretrained("sshleifer/bart-tiny-random")

bart_model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/bart-tiny-random")

In [ ]:
t5_base_tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-base")

t5_base_model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-v1_1-base")

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/605 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/945M [00:00<?, ?B/s]

In [ ]:
import gc

# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
train_dataset = MWPDataset(
                    df = train_df, 
                    lang = lang, 
                    context_col = ('Body', 'Question'), 
                    target_col = 'Equation',
                    tokenizer= pegasus_tokenizer,
                    MAX_LEN = MAX_LEN
                )
val_dataset = MWPDataset(
                    df = val_df, 
                    lang = lang, 
                    context_col = ('Body', 'Question'), 
                    target_col = 'Equation',
                    tokenizer= pegasus_tokenizer,
                    MAX_LEN = MAX_LEN
                )

In [ ]:
from transformers import Trainer, TrainingArguments, Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorWithPadding
from datasets import load_metric

import torch.nn.functional as F

# class MWPTrainer(Seq2SeqTrainer):
#   def compute_loss(self, model, inputs, return_outputs=False):
#     if self.label_smoother is not None and "labels" in inputs:
#         labels = inputs.pop("labels")
#     else:
#         labels = None
    
#     outputs = model(**inputs)
#     print(outputs)
#     print(outputs['logits'].shape, labels.shape)
#     loss = F.nll_loss(outputs['logits'], labels)
#     return (loss, outputs) if return_outputs else loss

rouge = load_metric('rouge')

training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    run_name="pegasus_base_1"
)

trainer = Trainer(
    model=pegasus_model, args=training_args, train_dataset=train_dataset, eval_dataset=val_dataset
)

Downloading:   0%|          | 0.00/2.17k [00:00<?, ?B/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train() 

***** Running training *****
  Num examples = 850
  Num Epochs = 10
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 8500


Step,Training Loss
500,2.326900
1000,1.454000
1500,1.353200
2000,1.223200
2500,1.232800
3000,1.152500
3500,1.130000
4000,1.066500
4500,1.080900
5000,1.018600


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2000
Configuration saved in ./results/checkpoint-2000/config.json
Model weights saved in ./results/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-2500
Configuration saved in ./results/checkpoint-2500/config.json
Model weights saved in ./results/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-3000
Configuration saved in ./results/checkpoint-3

TrainOutput(global_step=8500, training_loss=1.1704517929974725, metrics={'train_runtime': 3459.301, 'train_samples_per_second': 2.457, 'train_steps_per_second': 2.457, 'total_flos': 355454914314240.0, 'train_loss': 1.1704517929974725, 'epoch': 10.0})

In [ ]:
pred = trainer.predict(val_dataset)

***** Running Prediction *****
  Num examples = 150
  Batch size = 1


In [ ]:
eval = trainer.evaluate(val_dataset)

***** Running Evaluation *****
  Num examples = 150
  Batch size = 1


In [ ]:
eval

{'epoch': 10.0,
 'eval_loss': 0.11511595547199249,
 'eval_runtime': 2.0726,
 'eval_samples_per_second': 72.372,
 'eval_steps_per_second': 72.372}

In [ ]:
preds = np.argmax(pred.predictions[0], axis=2)

In [ ]:
pegasus_tokenizer.decode(preds[9])

In [ ]:
def clean_result(s):
  s = s.split("</s>")[0]
  return s

np.apply_along_axis((lambda x : clean_result(pegasus_tokenizer.decode(x))), 1, preds)

array(['( (2.0 * 15.0 )', '( 37.0 - 16.0 )', '( (0.0 - 1 10.0',
       '( 34.0 * 2 )', '( 6.0 + 6.0 )', '( 14.0 / 7.0 )',
       '( 10.0 - 11 11 ', '( 86.0 - 2 )', '( ( 12.0 + 13.0',
       '( 5 * 30.0 )', '( 148.0 - 103.0', '( 14.0 - 14.0 )',
       '( 28.0 / 2.0 )', '( 10.0 / 2.0 )', '( 33.0 - 2.06.0',
       '( 136.0 - 68.0 ', '( 27.0 - 20.0 )', '( 30.0 / 10.0 )',
       '( 39.0 - 7.0 )', '( 26.0 / 2.04.0', '( 18.0 - 1 )', '( 1 / 4.0 )',
       '( 10.0 - 15.01.', '( 13 138.0 - 13', '( ( 62.0 - 15.0',
       '( ( 55.0 + 555.', '( 10.00.0 / 10.', '( 4.04.0 - 2.00',
       '( 1 / 12.0 )', '( ( 38.0 - 1 ) ', '( ( - 2.0 )', '( 43.0 - 1 )',
       '( 5.0 + 6.0 )', '( 6.00.0 - 26.0', '( 6.0 * 2.0 )', '( 1 + 1 )',
       '( 56.0 - 4 )', '( 1 10.0 - 1 ) ', '( 1 1 + 1 ) - 4',
       '( 13.0 * 13.0 )', '( ( 28.0 + 34.0', '( 1 - ( )',
       '( 86.0 - 46.0 )', '( 14.0 - 7.02.0', '( 71.0 / 74.0 )',
       '( ( / 6.0 )', '( 1 16.0 / 6.06', '( 19.0 - 1 )', '( 5.0 + 3.0 )',
       '( 56.0 - 26.0 )'

In [ ]:
pegasus_tokenizer.decode(next(iter(val_dataset))['labels'])

In [ ]:
def correct(yhat, y):
  y = y.split("</s>")[0]
  yhat = yhat.split("</s>")[0]
  def strip(s):
    s = s.replace("<", "")
    s = s.replace(">", "")
    s = s.replace(" ", "")
    s = s.replace("(", "")
    s = s.replace(")", "")
    return s
  print(y, yhat)
  return strip(y) == strip(yhat)



In [ ]:
def eval_accuracy(tokenizer, preds):
  num_correct = 0
  total = 0 
  idx = 0

  val_dataset = MWPDataset(
                      df = val_df, 
                      lang = lang, 
                      context_col = 'Question', 
                      target_col = 'Equation',
                      tokenizer= tokenizer,
                      MAX_LEN = MAX_LEN
                  )

  yhats = np.apply_along_axis((lambda x : tokenizer.decode(x)), 1, preds)

  for q in val_dataset:
    y = tokenizer.decode(q["labels"])
    if correct(yhats[idx], y):
      num_correct += 1
    total += 1
    idx += 1

  return num_correct, total

In [ ]:
eval_accuracy(pegasus_tokenizer, preds)

In [ ]:
pegasus_model

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(60, 1024)
      (layers): ModuleList(
        (0): PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwi

In [ ]:
t5_base_model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo)

In [ ]:
t5_model

T5ForConditionalGeneration(
  (shared): Embedding(32102, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32102, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Dro